In [1]:
import eikon as ek
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
with open('key.txt') as f:
    ek.set_app_key(f.readline())

In [3]:
df_tickers = pd.read_csv("CompaniesSample.csv")

In [67]:
tickers_lst = df_tickers['Ticker'].to_list()

fin_vars_lst = [
    'TR.TotalReturn.date',
    'TR.PCTotAsset', 
    'TR.TotalEquity', 
    'TR.Revenue',
    'TR.NetProfitMargin',
    'TR.TotalDebtOutstanding',
    'TR.CapitalExpenditures',
    'TR.CurrentRatio'
    ]

In [68]:
df, err = ek.get_data(tickers_lst, fin_vars_lst, {'SDate': '2017-12-31', 'EDate': '2022-12-31', 'FRQ': 'FY', 'Curn': 'USD'})
df

,Instrument,Date,Total Assets (Pvt),Total Equity,Revenue,"Net Profit Margin, (%)",Total Debt,"Capital Expenditures, Cumulative",Current Ratio
0,CMPC.SN,2017-12-29T00:00:00Z,14950801000.0,8082116000.0,5143074000.0,2.0094,4082462000.0,-405076000.0,2.32273
1,CMPC.SN,2018-12-28T00:00:00Z,14983311000.0,8241835000.0,6274472000.0,8.00867,3808951000.0,-298923000.0,2.27579
2,CMPC.SN,2019-12-30T00:00:00Z,14955796000.0,8133310000.0,5670277000.0,1.49009,4065974000.0,-252058000.0,2.26496
3,CMPC.SN,2020-12-30T00:00:00Z,14726953000.0,7863289000.0,5286927000.0,-0.53653,4163993000.0,-167338000.0,2.99374
4,CMPC.SN,2021-12-30T00:00:00Z,15226299000.0,7546078000.0,6323022000.0,8.52142,4499243000.0,-224914000.0,2.06255
...,...,...,...,...,...,...,...,...,...
565,EPIRa.ST,2018-12-28T00:00:00Z,4084203153.94695,2123378970.67462,4324815869.14283,14.1831,653382133.658669,-117030409.832362,2.3406
566,EPIRa.ST,2019-12-30T00:00:00Z,4383358256.78274,2431211279.6411,4363277077.54753,14.37979,899807733.390301,-109271523.178808,3.07425
567,EPIRa.ST,2020-12-30T00:00:00Z,5342309003.2624,2884184642.35283,4397185567.51229,14.94657,1224132054.34095,-122340166.528704,3.64419
568,EPIRa.ST,2021-12-30T00:00:00Z,5374701301.00009,2846380210.63811,4385896982.03381,17.803,1016019116.73599,-102442694.043721,2.64528


In [93]:
df_fin = df_tickers[['Company Name', 'Ticker']].merge(df, right_on='Instrument', left_on='Ticker').drop(columns='Instrument')
df_fin['Date'] = df_fin['Date'].str[:-10] # Get rid of TZ info
df_fin = df_fin.loc[df_fin['Date'].str[:4].isin(['2017','2018','2019','2020','2021'])].reset_index(drop=True)

In [5]:
engine = create_engine("sqlite:///GDS.db")
df_fin.to_sql('FinancialsTableRaw', engine, index=False, if_exists="replace")

Calculations

In [6]:
df_fin = pd.read_sql_table('FinancialsTableRaw', engine)

In [7]:
df_fin['Net Profit Margin, (%)'] = df_fin['Net Profit Margin, (%)']/100
df_fin['Net Profit'] = df_fin['Net Profit Margin, (%)']*df_fin['Revenue']
df_fin['ROE'] = df_fin['Net Profit']/df_fin['Total Equity']

In [8]:
df_fin['Debt to Equity'] = df_fin['Total Debt']/df_fin['Total Equity']

In [9]:
df_fin['Capital Expenditures'] = df_fin['Capital Expenditures, Cumulative']/df_fin['Total Assets (Pvt)']*-1

In [10]:
df_fin.drop(columns=['Ticker', 'Total Equity', 'Total Assets (Pvt)', 'Total Debt', 'Capital Expenditures, Cumulative', 'Net Profit'], inplace=True)

In [11]:
df_fin

,Company Name,Date,Revenue,"Net Profit Margin, (%)",Current Ratio,ROE,Debt to Equity,Capital Expenditures
0,Empresas CMPC,2017-12-29,5.143074e+09,0.020094,2.32273,0.012787,0.505123,0.027094
1,Empresas CMPC,2018-12-28,6.274472e+09,0.080087,2.27579,0.060970,0.462148,0.019950
2,Empresas CMPC,2019-12-30,5.670277e+09,0.014901,2.26496,0.010388,0.499916,0.016854
3,Empresas CMPC,2020-12-30,5.286927e+09,-0.005365,2.99374,-0.003607,0.529549,0.011363
4,Empresas CMPC,2021-12-30,6.323022e+09,0.085214,2.06255,0.071403,0.596236,0.014771
...,...,...,...,...,...,...,...,...
459,Bureau Veritas,2021-12-31,5.662529e+09,0.084499,1.73766,0.256881,1.763320,0.017919
460,Epiroc AB,2018-12-28,4.324816e+09,0.141831,2.34060,0.288876,0.307709,0.028654
461,Epiroc AB,2019-12-30,4.363277e+09,0.143798,3.07425,0.258073,0.370107,0.024929
462,Epiroc AB,2020-12-30,4.397186e+09,0.149466,3.64419,0.227873,0.424429,0.022900


In [12]:
df_fin.to_sql('FinancialsTableFinal', engine, index=False, if_exists="replace")

464